In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from matplotlib import pyplot as plt

In [ ]:
DIR_INPUT = '/kaggle/input/global-wheat-detection'
DIR_TRAIN = f'{DIR_INPUT}/train'
DIR_TEST = f'{DIR_INPUT}/test'

DIR_WEIGHTS = '/kaggle/input/fasterrcnn-resnet50-fpnpth'

WEIGHTS_FILE = f'{DIR_WEIGHTS}/fasterrcnn_resnet50_fpn_20epochs.pth'

In [ ]:
train_df = pd.read_csv(f'{DIR_INPUT}/train.csv')
train_df.shape

In [ ]:
def unpack_bbox(df):
    '''
        Unpack bbox strings to int64
        
        Args:
            df(pd.DataFrame): A dataframe with a column named 'bbox',
                with strings of x,y,w,h of bounding boxes
        Returns:
            None
    '''
    df['x'] = -1
    df['y'] = -1
    df['w'] = -1
    df['h'] = -1

    def expand_bbox(x):
        r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
        if len(r) == 0:
            r = [-1, -1, -1, -1]
        return r

    df[['x', 'y', 'w', 'h']] = np.stack(df['bbox'].apply(lambda x: expand_bbox(x)))
    df.drop(columns=['bbox'], inplace=True)
    df['x'] = df['x'].astype(np.float32)
    df['y'] = df['y'].astype(np.float32)
    df['w'] = df['w'].astype(np.float32)
    df['h'] = df['h'].astype(np.float32)
    df['x'] = df['x'].astype(np.int32)
    df['y'] = df['y'].astype(np.int32)
    df['w'] = df['w'].astype(np.int32)
    df['h'] = df['h'].astype(np.int32)
    df['x1'] = df['x']
    df['y1'] = df['y']
    df['x2'] = df['x']+df['w']
    df['y2'] = df['y']+df['h']

In [ ]:
unpack_bbox(train_df)
train_df.head()

In [ ]:
eval_df = pd.read_csv(f'/kaggle/input/wheat-eval/fasterrcnn_resnet50_fpn_10epochs_results.csv')
unpack_bbox(eval_df)
eval_df.head()

In [ ]:
def bboxes_one_img(train_df,eval_df,image_id):
    labelled_bboxes = train_df[train_df['image_id']==image_id]
    pred_bboxes = eval_df[eval_df['image_id']==image_id]
    return labelled_bboxes,pred_bboxes

In [ ]:
def visualize_result(image,pred_bboxes,pred_scores=None,labelled_bboxes=None):
    '''
        Visualize predicted bboxes with optional scores and optional ground truth bboxes
        
        Args:
            image(np.array): an image of wheat
            pred_bboxes(pd.DataFrame(int)): 
                a dataframe of columns of predicted bboxes x1,y1,x2,y2 coordinates
            pred_scores(pd.Series or [], optional): a list of confidence of predicted bboxes
            labelled_bboxes(pd.DataFrame(int), optional): 
                a dataframe of columns of ground truth bboxes x1,y1,x2,y2 coordinates
        Returns:
            None
    '''
    
    fig, ax = plt.subplots(1, 1, figsize=(16, 8))
    boxes = pred_bboxes
    for i in range(len(boxes)):
        cv2.rectangle(image,
                      (boxes['x1'][i].astype(np.int64), boxes['y1'][i].astype(np.int64)),
                      (boxes['x2'][i].astype(np.int64), boxes['y2'][i].astype(np.int64)),
                      color = (0,220, 0), thickness = 3)
        if pred_scores is not None:
            cv2.putText(image, f'{pred_scores[i]:.2f}', (boxes['x1'][i].astype(np.int64), 
                        boxes['y1'][i].astype(np.int64)-10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, (36,255,12), 2)
            
    if labelled_bboxes is not None:
        boxes = labelled_bboxes
        for i in range(len(boxes)):
            cv2.rectangle(image,
                          (boxes['x1'][i].astype(np.int64), boxes['y1'][i].astype(np.int64)),
                          (boxes['x2'][i].astype(np.int64), boxes['y2'][i].astype(np.int64)),
                          color = (220, 0, 0), thickness = 3)

    ax.set_axis_off()
    ax.imshow(image)

In [ ]:
image_ids = train_df['image_id'].unique()
image_id = image_ids[0]
labelled_bboxes,pred_bboxes = bboxes_one_img(train_df,eval_df,image_id)

image = cv2.imread(f'{DIR_TRAIN}/{image_id}'+'.jpg')
pred_scores = pred_bboxes['score']
visualize_result(image,pred_bboxes,pred_scores,labelled_bboxes)

In [ ]:
# image = cv2.imread(f'{DIR_TRAIN}/{image_id}'+'.jpg')
sharpen_kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
edge_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])

In [ ]:
image = cv2.imread(f'{DIR_TRAIN}/{image_id}'+'.jpg')
image = cv2.filter2D(image, -1, sharpen_kernel)
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
ax[0].set_axis_off()
ax[0].imshow(image)
image = cv2.imread(f'{DIR_TRAIN}/{image_id}'+'.jpg')
image = cv2.filter2D(image, -1, edge_kernel)
ax[1].set_axis_off()
ax[1].imshow(image)